In [88]:
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pandas as pd
import numpy as np
import re

def rename(text): return re.sub(r'[^a-zA-Z]', "", text)

def compute_one_way_anova(df, numerical_columns, group_cols):
    results = []
    original_group_cols = group_cols[:]
    group_cols = [rename(col) for col in group_cols]
    df = df.rename(columns={col: rename(col) for col in df.columns})
    
    for original_group, group in zip(original_group_cols, group_cols):
        for col in numerical_columns:
            column_name = rename(col)  
            formula = f"{column_name} ~ C({group})" 
            model = smf.ols(formula, data=df).fit()
            anova_table = sm.stats.anova_lm(model, typ=2)
            
            for source, row in anova_table.iterrows():
                p_value = row["PR(>F)"]
                interpretation = "Significant" if p_value < 0.05 else "No significant"
                if source == "Residual": interpretation = "-"
                
                results.append({
                    "Group": original_group,
                    "Variable": col,
                    "Source": source,
                    "Sum Sq": row["sum_sq"],
                    "df": row["df"],
                    "F-Value": row["F"],
                    "p-Value": p_value,
                    "Significant (α<0.05)": interpretation
                })
    results = pd.DataFrame(results)
    return results.fillna(' ')

if __name__ == "__main__":
    df = pd.read_csv("../Datasets/Eggplant Fusarium Fertilizer Data.csv")
    group_columns = ['Fertilizer', 'Variety']
    variables = df.select_dtypes(include=[np.number]).columns  
    anova_results = compute_one_way_anova(df, variables, group_columns)
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    display(anova_results)

,Group,Variable,Source,Sum Sq,df,F-Value,p-Value,Significant (α<0.05)
0,Fertilizer,Replication ID,C(Fertilizer),0.0000,2.0000,0.0000,1.0000,No significant
1,Fertilizer,Replication ID,Residual,666600.0000,797.0000,,,-
2,Fertilizer,Infection Severity (%),C(Fertilizer),431897.2417,2.0000,3148.1180,0.0000,Significant
3,Fertilizer,Infection Severity (%),Residual,54671.0933,797.0000,,,-
4,Fertilizer,Wilt index,C(Fertilizer),1704.0082,2.0000,2559.7159,0.0000,Significant
5,Fertilizer,Wilt index,Residual,265.2823,797.0000,,,-
6,Fertilizer,Plant height (cm),C(Fertilizer),59164.8267,2.0000,1029.4647,0.0000,Significant
7,Fertilizer,Plant height (cm),Residual,22902.3725,797.0000,,,-
8,Fertilizer,Days to wilt symptoms,C(Fertilizer),19281.7021,2.0000,3800.0845,0.0000,Significant
9,Fertilizer,Days to wilt symptoms,Residual,2021.9967,797.0000,,,-
